# 01- Intro. Homework

## Q1. Downloading the data

We'll use the same [NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Load DataFrame from Parquet file
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df.sample(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1820704,2,2023-01-20 08:42:12,2023-01-20 08:49:54,2.0,1.10,1.0,N,237,161,1,9.3,0.0,0.5,1.00,0.0,1.0,14.30,2.5,0.00
1059391,2,2023-01-12 18:19:27,2023-01-12 18:30:23,1.0,1.63,1.0,N,68,224,1,11.4,2.5,0.5,3.58,0.0,1.0,21.48,2.5,0.00
1677024,2,2023-01-18 21:20:47,2023-01-18 21:44:37,4.0,12.42,5.0,N,138,265,1,115.0,5.0,0.0,28.06,19.3,1.0,169.61,0.0,1.25


In [3]:
# Calculate number of columns
print(f'the number of columns in January is: {len(df.columns)}')

the number of columns in January is: 19


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [4]:
# Create 'duration' column
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [5]:
# Convert to minutes
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
df.sample(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
697855,2,2023-01-08 22:13:00,2023-01-08 22:30:33,2.0,4.31,1.0,N,230,13,1,22.6,1.0,0.5,5.52,0.00,1.0,33.12,2.5,0.0,17.550000
1856493,2,2023-01-20 16:26:03,2023-01-20 17:29:26,1.0,17.57,2.0,N,162,132,1,70.0,5.0,0.5,17.11,6.55,1.0,102.66,2.5,0.0,63.383333
1217294,1,2023-01-14 06:17:41,2023-01-14 06:23:51,2.0,1.60,1.0,N,48,239,2,9.3,2.5,0.5,0.00,0.00,1.0,13.30,2.5,0.0,6.166667


In [7]:
# Deviation standard of duration
std_duration = df['duration'].std()
print(f'the standard deviation of the trips in January is: {round(std_duration,2)}')

the standard deviation of the trips in January is: 42.59


## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [8]:
# Check the distribution of the duration
print(df['duration'].describe())

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64


In [9]:
# Filter records with duration between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

In [10]:
# Check duration distribution after filtering
print(df_filtered['duration'].describe())

count    3.009173e+06
mean     1.420486e+01
std      9.939386e+00
min      1.000000e+00
25%      7.216667e+00
50%      1.155000e+01
75%      1.818333e+01
max      6.000000e+01
Name: duration, dtype: float64


In [11]:
# Calculate the fraction of records remaining
fraction_remaining = len(df_filtered) / len(df)
print(f'Fraction of records left after dropping outliers: {round(fraction_remaining*100)}%')

Fraction of records left after dropping outliers: 98%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?


In [12]:
# Cast the pickup and dropoff location IDs to strings
df_filtered['PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered['DOLocationID'] = df_filtered['DOLocationID'].astype(str)

In [13]:
# Turn the DataFrame into a list of dictionaries
df_dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [14]:
# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(df_dicts)

In [15]:
# Check the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]
print(f'The dimensionality of the feature matrix (number of columns): {num_columns}')

The dimensionality of the feature matrix (number of columns): 515


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [16]:
# Prepare the target variable (duration)
y_train = df_filtered['duration'].values

In [17]:
# Train a plain linear regression model with default parameters
model = LinearRegression()
model.fit(feature_matrix, y_train)

LinearRegression()

In [19]:
# Predict the target variable on the training data
y_train_pred = model.predict(feature_matrix)

In [20]:
# Calculate the RMSE on the training data
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f'RMSE on training data: {rmse_train:.2f}')

RMSE on training data: 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [21]:
# Load the validation DataFrame from the Parquet file
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

# Create the 'duration' column in minutes using vectorized operations
df_val['duration'] = (df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime).dt.total_seconds() / 60

# Filter records with duration between 1 and 60 minutes (inclusive)
df_val_filtered = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)].copy()

# Cast the pickup and dropoff location IDs to strings
df_val_filtered['PULocationID'] = df_val_filtered['PULocationID'].astype(str)
df_val_filtered['DOLocationID'] = df_val_filtered['DOLocationID'].astype(str)

# Turn the validation DataFrame into a list of dictionaries
data_dicts_val = df_val_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [22]:
# Transform the validation data using the fitted DictVectorizer
feature_matrix_val = dv.transform(data_dicts_val)

# Prepare the target variable (duration) for validation data
y_val = df_val_filtered['duration'].values

In [23]:
# Predict the target variable on the validation data
y_val_pred = model.predict(feature_matrix_val)

In [24]:
# Calculate the RMSE on the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f'RMSE on validation data: {rmse_val:.2f}')

RMSE on validation data: 7.81
